In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

In [8]:
schema = StructType([
    StructField("vid", StringType(), True),
    StructField("text", StringType(), True),
    StructField("likes", IntegerType(), True),
])

df = spark.read.schema(schema).format("csv").option("header",True).option("encoding", "gbk").load("./xx/test.csv")
df = df.orderBy(desc(col("likes")))

In [9]:
df.show()

+-----------+----------------------+-----+
|        vid|                  text|likes|
+-----------+----------------------+-----+
|_4cXJKUunKk|  The way this pand...|  177|
|_4cXJKUunKk|  Most care homes a...|  152|
|_4cXJKUunKk|  the good news is ...|   98|
|_4cXJKUunKk|  The virus is show...|   94|
|_4cXJKUunKk|  This is unbelieva...|   70|
|_4cXJKUunKk|  A second wave?<br...|   70|
|_4cXJKUunKk|I鈥檓 a healthcare ...|   61|
|_4cXJKUunKk|  "Those of us who ...|   52|
|_4cXJKUunKk|  We should look af...|   43|
|_4cXJKUunKk|  Boris ignored the...|   36|
|_4cXJKUunKk|  My condolences fo...|   29|
|_4cXJKUunKk|  This is shameful....|   28|
|_4cXJKUunKk|  The government ha...|   26|
|_4cXJKUunKk|  get your parents ...|   25|
|_4cXJKUunKk|  WTF where are all...|   21|
|_4cXJKUunKk|  Jesus Christ! We ...|   20|
|_4cXJKUunKk|  The Government be...|   20|
|_4cXJKUunKk|  Remember folks, t...|   19|
|_4cXJKUunKk|  oh I am crying RI...|   19|
|_4cXJKUunKk| I feel so sorry 馃...|   18|
+-----------+-